In [1]:
import numpy as np
import statsmodels.api as sm

In [2]:
def gen_data(nobs, num_cov, a):
    xn = np.random.normal(scale=1., size=(nobs, num_cov))
    x0 = np.random.normal(scale=1., size=(nobs, 1))
    x0 = np.tile(x0,2)
    xn = xn +x0
    e = np.random.normal(loc=0.0, scale=1.0, size=(nobs))
    yn = a + (xn.sum(axis=1) * a) + e
    return yn, xn

y,x = gen_data(1000,2,1)
print(y.shape,x.shape)

(1000,) (1000, 2)


In [3]:
model0 = sm.OLS(y,sm.add_constant(x)).fit()
print('r2 true', model0.fittedvalues.var()/y.var() )

r2 true 0.8666374049062794


In [4]:
model1 = sm.OLS(y,sm.add_constant(x[:,0])).fit()
print('r2 1',model1.fittedvalues.var()/y.var())
model2 = sm.OLS(y,sm.add_constant(x[:,1])).fit()
print('r2 2',model2.fittedvalues.var()/y.var())

r2 1 0.659306083417603
r2 2 0.6548200239805115


approach 1: regress error 1 on predicted 2

In [10]:
## THIS IS THE right way... it seems better.. 
error_reg = sm.OLS(model1.resid,sm.add_constant(model2.fittedvalues) ).fit()

#split up unexplained varation into 2 pieces... 
print(model1.resid.var())
print(error_reg.resid.var() + error_reg.fittedvalues.var())

#not quite the same? as when i do it with correlations?
print((error_reg.fittedvalues.var())/y.var()  ) #is this the additional r2 that y2 can add?
print( (model1.fittedvalues.var() + error_reg.fittedvalues.var())/y.var()  )
print( (model1.fittedvalues.var() + error_reg.fittedvalues.var())/y.var()  )

2.3220755470433443
2.322075547043345
0.15205106170826616
0.8113571451258691
0.8113571451258691


approach 2: covariance matrices

In [6]:
#covariance matrix approach
cov = np.cov( [y,model1.fittedvalues,model2.fittedvalues] )
print(cov)

#double check that diagnals are r2
print(1- (y - model1.fittedvalues).var()/y.var())
print(cov[1,1]/cov[0,0])

#what is the r2 of model 2
print(1- (y - model2.fittedvalues).var()/y.var())
print(cov[2,2]/cov[0,0])

[[6.82254609 4.49814614 4.46753979]
 [4.49814614 4.49814614 2.31474724]
 [4.46753979 2.31474724 4.46753979]]
0.6593060834176026
0.6593060834176029
0.6548200239805111
0.6548200239805112


In [7]:
#how much could the r2 of model 1 be-improved? what is the r2 of 1 and what is the correlation?
beta = (cov[0,2] - cov[1,2])/(cov[2,2])
print(beta)

print( (beta**2*cov[2,2])/cov[0,0]) #this is it!!

#can simplify
print( (cov[0,2] - cov[1,2])**2/cov[2,2]/cov[0,0])  #this is it!!
#TODO think this through using variance/covariance formula

#how far is model 1 from the true model, using model 2 as a yard stick... 
#distance of 1 from the truth, relative to unexplained error

0.4818742870000738
0.15205106170826593
0.15205106170826593


relationship with infomrativeness

In [8]:
## THIS IS THE right way... it seems better.. 
error_reg = sm.OLS( model1.resid,sm.add_constant(model2.fittedvalues) ).fit()

#informativesness about model 2 for model 1 residuals
print(error_reg.fittedvalues.var()/model1.resid.var())

#not quite the same? as when i do it with correlations?
print( (cov[0,2] - cov[1,2])**2/(cov[2,2]*(cov[0,0]-cov[1,1]) ) )

0.44629814125692585
0.44629814125692563
